<a href="https://colab.research.google.com/github/PranavRathod399/Blacklace/blob/main/code/blacknet/symptom_triage_analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re

# Base risk scores
symptom_score = {
    "fever": 1,
    "cough": 1,
    "sore throat": 1,
    "shortness of breath": 2,
    "blood": 3,
    "chest pain": 3,
    "headache": 1,
    "nausea": 1,
    "vomiting": 2,
    "diarrhea": 2,
}

# Extract symptoms and duration
def extract_symptoms_and_duration(text):
    text = text.lower()
    symptoms_found = [symptom for symptom in symptom_score if symptom in text]

    # Try various expressions for duration
    patterns = [
        r"for (\d+ (day|days|week|weeks))",
        r"for (a few|few|several) (days|weeks)",
        r"since (yesterday|last night|monday|tuesday|wednesday|thursday|friday|saturday|sunday)",
        r"since (a few|few|several) (days|weeks)",
        r"(over|during) the past (\d+ (day|days|week|weeks))"
    ]

    duration = None
    for pattern in patterns:
        match = re.search(pattern, text)
        if match:
            duration = match.group(0)
            break

    return symptoms_found, duration

# Triage logic
def assess_risk(symptoms, duration=None):
    total_score = sum(symptom_score.get(symptom, 0) for symptom in symptoms)

    if "cough" in symptoms and "blood" in symptoms:
        return "Emergency care advised"
    if "fever" in symptoms and "shortness of breath" in symptoms:
        return "Consult a doctor"

    if duration:
        if "week" in duration.lower() or "7 days" in duration.lower():
            total_score += 1

        if total_score >= 5:
            return "Consult a doctor"
        elif total_score >= 2:
            return "Monitor at home"
        elif total_score > 0:
            return "Mild: Home care"
        else:
            return "Insufficient data for triage"

# Full pipeline
def analyze_input(text):
    symptoms, duration = extract_symptoms_and_duration(text)
    triage = assess_risk(symptoms, duration)
    return {
        "symptoms": symptoms,
        "duration": duration,
        "triage": triage
    }

In [15]:
# Optional interactive test -version 2
if __name__ == "__main__":
    user_input = input("Describe your symptoms: ")
    result = analyze_input(user_input)

    print("\n--- Symptom Analysis ---")
    print("Symptoms Detected :", ", ".join(result["symptoms"]) if result["symptoms"] else "None")
    print("Duration           :", result["duration"] or "Not specified")
    print("Triage Assessment  :", result["triage"])

Describe your symptoms: I got a really bad headache fever and blood cough since Monday 

--- Symptom Analysis ---
Symptoms Detected : fever, cough, blood, headache
Duration           : since monday
Triage Assessment  : Emergency care advised
